# Observing steps per second

In [1]:
import wandb
import json
import os
import pickle
from collections import defaultdict
import pandas as pd

In [2]:
# glob.glob("runs/**/events.out.tfevents*")

In [3]:
wandb.login()

# Define where to store the cache files
cache_dir = "./wandb_cache/"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

wandb: Currently logged in as: adrian-orenstein (the-orbital-mind). Use `wandb login --relogin` to force relogin


In [4]:
project_name = "beluga_8cpu-testing-lunarlander-v2-300000"
entity_name = "the-orbital-mind"  # Optionalimport pandas as pd

In [5]:
# Fetch runs
api = wandb.Api()
runs = api.runs(f"{entity_name}/{project_name}")

# Check if there are any runs available
if runs:
    # Fetch history of the first run
    first_run = runs[0]
    history_df = first_run.history()

    # Print available keys in the history
    print("Available keys in the history:")
    for key in history_df.columns:
        print(key)
else:
    print("No runs found.")


Available keys in the history:
environment/ratio
_step
environment/num_repeat_actions
agent/step_sps
_runtime
environment/agent_response_time
global_step
_timestamp
agent/step_dt
charts/episodic_length
charts/episodic_return
agent_losses/td_loss
agent_losses/q_values
dqn/update_target_network


In [6]:
import os
import pickle
import wandb
from concurrent.futures import ThreadPoolExecutor

PROJECTS = [
    "beluga_8cpu-testing-lunarlander-v2-300000",
    "beluga_8cpu-testing-acrobot-v1-300000",
    "beluga_8cpu-testing-mountaincar-v0-300000",
    "beluga_8cpu-testing-cartpole-v1-300000",
]

for project_name in PROJECTS:
    api = wandb.Api(timeout=20)
    runs = api.runs(f"{entity_name}/{project_name}")
    
    # Define where to store the cache files
    cache_dir = "./wandb_cache/"
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    
    def process_run(job):
        (i, run) = job
    
        cache_path = os.path.join(cache_dir, f"{run.id}.pkl")
    
        # Check if we have cached data
        if not os.path.exists(cache_path):
            df = run.history()
            with open(cache_path, 'wb') as file:
                pickle.dump(df, file)
            print(f"Data cached for run: {run.id}, {i/len(runs):0.2}")
        # else:
        #     print(f"Already cached run: {run.id}, {i/len(runs):0.2}")
    
    # Use ThreadPoolExecutor to handle the runs in parallel
    with ThreadPoolExecutor(max_workers=8) as executor:
        executor.map(process_run, (enumerate(runs)))


Data cached for run: tnfhwg13, 0.0054
Data cached for run: aofrrlmu, 0.0
Data cached for run: 0uv7d7iz, 0.011
Data cached for run: c9phhoud, 0.022
Data cached for run: htg7nbmp, 0.038
Data cached for run: 7bxz561k, 0.016
Data cached for run: y5hjgko8, 0.043
Data cached for run: di0c1bsn, 0.032
Data cached for run: x98jzqyt, 0.049
Data cached for run: z761ptct, 0.059
Data cached for run: m5eue7nr, 0.027
Data cached for run: ksqtqbya, 0.054
Data cached for run: fk6ezf7c, 0.07
Data cached for run: nstq2fln, 0.065
Data cached for run: bzc9jltj, 0.076
Data cached for run: 7ttfiuip, 0.081
Data cached for run: ro31cqmb, 0.092
Data cached for run: bjawfn9p, 0.097
Data cached for run: qjdxf8fl, 0.086
Data cached for run: aslwrz7n, 0.1
Data cached for run: zru2eb6t, 0.11
Data cached for run: 7p91n8h8, 0.11
Data cached for run: 66qd9jbv, 0.12
Data cached for run: biuwcb4m, 0.13
Data cached for run: 1i82yc1d, 0.14
Data cached for run: 2x5a3gv9, 0.14
Data cached for run: cozvznnt, 0.15
Data cached 

In [7]:
len(runs)

202

In [8]:
from collections import defaultdict
data = defaultdict(dict)

for project_name in PROJECTS:
    api = wandb.Api(timeout=20)
    runs = api.runs(f"{entity_name}/{project_name}")
    data[project_name] = defaultdict(list)
    for run in runs:
        cache_path = os.path.join(cache_dir, f"{run.id}.pkl")
        if run.state != "finished":
            # print(cache_path)
            continue
            
        config = {k:v.get('value') for k, v in json.loads(run.json_config).items()}
    
        if config.get('async_datarate') is None:
            continue
    
        if os.path.exists(cache_path):
            run_df = pd.read_pickle(cache_path)
        else:
            run_df = run.history()
            run_df.to_pickle(cache_path)
    
        df = run_df[['charts/episodic_return']]
    
        dic = dict(df[~df['charts/episodic_return'].isnull()].mean())
    
        data[project_name][config.get('async_datarate')].append(dic.get('charts/episodic_return'))


KeyError: "None of [Index(['charts/episodic_return'], dtype='object')] are in the [columns]"

In [ ]:
data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
renamed_data = {
    "Cluster, DQN, lunarlander-v2": data["beluga_8cpu-testing-lunarlander-v2-300000"],
    "Cluster, DQN, acrobot-v1": data["beluga_8cpu-testing-acrobot-v1-300000"],
    "Cluster, DQN, mountaincar-v0": data["beluga_8cpu-testing-mountaincar-v0-300000"],
    "Cluster, DQN, cartpole-v1": data["beluga_8cpu-testing-cartpole-v1-300000"],
}

# {
#     "beluga dqn acrobot-v0": data['beluga_4cpu_perf_arrayjob-simplified-async-interface-with-dqn-acrobot-v1-None'],
#     "beluga dqn cartpole-v1": data['beluga_4cpu_perf_arrayjob-simplified-async-interface-with-dqn-mountaincar-v0-None'],
#     "beluga dqn cartpole-v1": data['beluga_4cpu_perf_arrayjob-simplified-async-interface-with-dqn-cartpole-v1-None'],
#     "m1 dqn cartpole-v1": data['m1_mac-simplified-async-interface-with-dqn-cartpole-v1-None'],
# }


In [ ]:
import numpy as np
import math
for project_name, project_data in renamed_data.items():
    # Convert the data into a DataFrame suitable for Seaborn
    data_points = []
    seed_count = []
    for step_rate, values in project_data.items():
        seed_count.append(len(values))
        for value in values:
            data_points.append({"Environment Step Rate": step_rate, "Mean Episodic Return": value})
    
    df = pd.DataFrame(data_points)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.lineplot(x="Environment Step Rate", y="Mean Episodic Return", data=df, errorbar=('ci', 95), ax=ax)
    ax.set_title(project_name + f" performance by environment step rate, seeds={math.floor(np.array(seed_count).mean())}")
    ax.set_xlabel("Environment Step Rate")
    ax.set_ylabel("Mean Episodic Return")
    labels = sorted(df['Environment Step Rate'].unique())
    ax.set_xticks(labels, rotation=45, ha='right', rotation_mode='anchor')
    ax.grid(True)
    
    fig.savefig(f'performance_by_steprate_{project_name.replace(", ","")}.svg', format="svg")